# Data Warngling Project: Wikipedia List of Roman Emperors

##### Import necessary modules

This Jupyter Notebook loads a Wikipedia list of Roman emperors into a Pandas DataFrame and then cleans and transforms the data. 

The original Wikipedia list includes Roman and Byzantine emperors. The project will only cover the period up to the fall of the Western Roman empire in 476. The last emperor included in the project data is Romulus Augustulus.

The original Wikipedia list consists of five columns:
1. Portrait
2. Name
3. Reign
4. Succession
5. Life details

Two columns have been added during the web scraping of the Wikipedia page:
1. Period
2. Epoch

During the project the originals columns will be split and the relevant information extracted.

The final dataset consists of 33 columns:
1. Common Name
2. Full Name
3. Certainty Begin First Reign
4. Era Begin First Reign
5. Begin First Reign
6. Certainty End First Reign
7. End First Reign
8. Days of First Reign
9. Years of First Reign
10. Certainty Begin Second Reign
11. Begin Second Reign
12. Certainty End Second Reign
13. End Second Reign
14. Days of Second Reign
15. Years of Second Reign
16. Part of Empire
17. Period
18. Epoch
19. Legitimacy
20. Status as Emperor
21. Succession 1
22. Succession 2
23. Succession 3
24. Certainty Birth
25. Era Birth
26. Date of Birth
27. Certainty Death
28. Date of Death
29. Age
30. Life details 1
31. Life details 2 
32. Life details 3
33. Life details 4

In [144]:
# Import pandas, numpy and re module.
import pandas as pd
import numpy as np
import re

##### Load the List of Roman Emperors

In [145]:
# Load list of Roman emperors.
# The targeted Wikipedia page is: https://en.wikipedia.org/wiki/List_of_Roman_emperors
# The data has been pre-downloaded into a csv-file: 
emperors = pd.read_csv(r"data_roman_emperors.csv")

##### Drop empty columns and rows. Remove dashes and footnotes

In [146]:
# Drop the 'Portrait' column, because it only contains 'Null'-values.
emperors = emperors.drop('Portrait', axis=1)

In [147]:
# Drop empty rows.
emperors = emperors.dropna()

In [148]:
# Reset index.
emperors = emperors.reset_index(drop=True)

In [149]:
# Replace dash with hyphen.
emperors = emperors.replace(to_replace="–", value="-", regex=True)


In [150]:
# Some values have footnotes in square brackets attached to them, like '[m]', '[r]' or '[34]'.
# Remove all footnotes in all columns.
emperors = emperors.replace(to_replace=r"\[(.|..|...)]", value="", regex=True)

# 'Name' column

##### Clean 'Name' column

In [151]:
# Remove quotation marks and parenthesis. 
emperors['Name'] = emperors['Name'].replace(to_replace='"', value='', regex=True)
emperors['Name'] = emperors['Name'].replace(to_replace='\\(', value='', regex=True)
emperors['Name'] = emperors['Name'].replace(to_replace='\\)', value='', regex=True)

##### Remove symbols from 'Name' column and perserve information in new columns 'Legitimacy' and 'Status as Emperor'

It's not always clear who was the Roman emperor at a given time. The Wikipedia page marks unclear cases with two symbols in the 'Name' column:

- (#) 'Ambiguous legitimacy' = cases where more than one emperor held power at a given time and the recorded emperor did not achieved the full recognition of the other emperor(s).

- (§) 'Varying ascribed status' = cases where an emperor ascended the throne as a child and is therfor not recognozied by all scholars.

In [152]:
# Find rows that contain symbols (#) and (§).
legitimacy_condition = emperors["Name"].str.match(r".*(#).*")
status_condition = emperors["Name"].str.match(r".*(§).*")

# Create a new columns 'Legitimacy' with values "weak" (marked by symbol) and "strong".
emperors["Legitimacy"] = np.where(legitimacy_condition, "weak", "strong")

# Create a new columns 'Status as Emperor' with values "controversial" (marked by symbol) and "canonical".
emperors["Status as Emperor"] = np.where(status_condition, "controversial", "canonical")

# Replace symbols '(#)' and '(§)' in the 'Name' column.
emperors["Name"].replace(to_replace=r"\s#", value="", regex=True, inplace=True)
emperors["Name"].replace(to_replace=r"\s§", value="", regex=True, inplace=True)

#### Split 'Name' column into new columns 'Common Name' and 'Full Name'

Roman naming conventions are complex. A person's name had usually three parts: nomen, praenomen and cognomen. This naming convention began to fade away from the 3rd century onwards. As a consequence the names of the Roman emperors vary in length and do not follow a common pattern. Usually emperors have long official names, but are commonly known under a shorter name. The common name can consist of one, two, three or four words. 

In [153]:
# Create lists with the emperors' common names, differentiating between two, three and four word names.
four_word_names = ['Constantine I the Great', 'Valentinian I the Great', 'Theodosius I the Great', 
                   'Philip I the Arab', 'Philip II the Younger']

three_word_names = ['Maximinus I Thrax', 'Maximinus II Daza', 'Claudius II Gothicus', 
                    'Julian the Apostate', 'Constantius I Chlorus']

two_word_names = ['Antoninus Pius', 'Marcus Aurelius', 'Lucius Verus', 'Didius Julianus', 
                  'Septimius Severus', 'Severus Alexander', 'Gordian I', 'Gordian II', 'Gordian III', 
                  'Herennius Etruscus', 'Trebonianus Gallus', 'Diocletian Jovius', 'Maximian Herculius', 
                  'Severus II', 'Valerius Valens', 'Constantine II', 'Constantius II', 'Magnus Maximus', 
                  'Valentinian II', 'Constantine III', 'Theodosius II', 'Priscus Attalus', 'Constantius III', 
                  'Valentinian III', 'Petronius Maximus', 'Libius Severus', 'Julius Nepos', 'Romulus Augustulus']

# Create new columns 'Common Name and 'Full Name'.
emperors['Common Name'] = emperors['Name']
emperors['Full Name'] = emperors['Name']

# Split 'Name' column and copy first part (corresponds to emperor names with only one word) into the 'Common Name' column.
# Copy 'Name' column without the 'Common Name' part into 'Full Name' column. 
for i in range(len(emperors)):
    name_split = emperors['Name'][i].split(" ")
    new_name = name_split[0]
    emperors['Common Name'][i] = new_name
    emperors['Full Name'][i] = emperors['Name'][i].replace((new_name + " "), "")

    # Check if longer parts of the split are in the lists. If so, copy the correspondig name into the 'Common Name' column.
    new_name = " ".join(name_split[:2])
    if new_name in two_word_names:
        emperors['Common Name'][i] = new_name
        emperors['Full Name'][i] = emperors['Name'][i].replace((new_name + " "), "")
    new_name = " ".join(name_split[:3])
    if new_name in three_word_names:
        emperors['Common Name'][i] = new_name
        emperors['Full Name'][i] = emperors['Name'][i].replace((new_name + " "), "")
    new_name = " ".join(name_split[:4])
    if new_name in four_word_names:
        emperors['Common Name'][i] = new_name
        emperors['Full Name'][i] = emperors['Name'][i].replace((new_name + " "), "")

# 'Reign' column

The 'Reign' column contains five types of information:
- the dates of the first reign
- the dates of the second reign
- the duration of the first reign
- the duration of the second reign
- information about the controlled territory

##### Extract reign information from 'Reign' column and store it in new columns 'Begin First Reign', 'End First Reign', 'Begin Second Reign' and 'End Second Reign'

Most emperors ruled continously without interruption. But the reign of some emperors was interrupted, creating a first and a second reign of these emperors.

In [154]:
# Split 'Reign' column into new columns 'First Reign' and 'Second Reign'.
emperors['First Reign'] = emperors['Reign'].str.extract(pat=r"(.*?)(?=\s\()", expand=True)
emperors['Second Reign'] = emperors['Reign'].str.extract(pat=r"(?<=\)\s)(.*)(?=\s\()", expand=True)

In [155]:
# Split 'First Reign' into new columns 'Begin First Reign', 'End First Reign'.
emperors['Begin First Reign'] = emperors['First Reign'].str.extract(pat=r"(.*)(?=\s\-)", expand=True)
emperors['End First Reign'] = emperors['First Reign'].str.extract(pat=r"(?<=\-\s)(.*)", expand=True)

# Split 'Second Reign' into new columns 'Begin Second Reign', 'End Second Reign'.
emperors['Begin Second Reign'] = emperors['Second Reign'].str.extract(pat=r"(.*)(?=\s\-)", expand=True)
emperors['End Second Reign'] = emperors['Second Reign'].str.extract(pat=r"(?<=\-\s)(.*)", expand=True)

In [156]:
# The 'Reign' column has been split using a hyphen '-' as demiliter. 
# But not all rows of the 'Reign' column contain a hyphen. 
# These rows are still empty.
# Fill empty rows.
begin_first_reign_null = emperors['Begin First Reign'].isnull()
end_first_reign_null = emperors['End First Reign'].isnull()
begin_second_reign_null = emperors['Begin Second Reign'].isnull()
end_second_reign_null = emperors['End Second Reign'].isnull()

emperors.loc[begin_first_reign_null, 'Begin First Reign'] = emperors['First Reign']
emperors.loc[end_first_reign_null, 'End First Reign'] = emperors['First Reign']
emperors.loc[begin_second_reign_null, 'Begin Second Reign'] = emperors['Second Reign']
emperors.loc[end_second_reign_null, 'End Second Reign'] = emperors['Second Reign']

In [157]:
# Replace trailing white spaces.
emperors['Begin First Reign'] = emperors['Begin First Reign'].replace(to_replace=r"\s$", value="", regex=True)
emperors['End First Reign'] = emperors['End First Reign'].replace(to_replace=r"\s$", value="", regex=True)
emperors['Begin Second Reign'] = emperors['Begin Second Reign'].replace(to_replace=r"\s$", value="", regex=True)
emperors['End Second Reign'] = emperors['End Second Reign'].replace(to_replace=r"\s$", value="", regex=True)

In [158]:
# The reign of emperor Augustus started befor the year 0 (marked with 'BC') and ended after the year 0 (marked with 'AD'). 
# All other emperors reigned after the year 0.
# This is only an issue for the column 'Begin First Reign'.
# Store information in an column 'Era Begin First Reign'.
def determine_era(row_value):
    input_string = str(row_value)
    if re.search(r"\sBC", input_string):
        return "BC"
    else:
        return "AD"

emperors['Era Begin First Reign'] = emperors['Begin First Reign'].apply(determine_era)

# Remove 'BC' and 'AD' from the 'Begin First Reign' and the 'End First Reign' column.
emperors['Begin First Reign'] = emperors['Begin First Reign'].replace(to_replace=r"\sBC", value="", regex=True)
emperors['End First Reign'] = emperors['End First Reign'].replace(to_replace=r"\sAD", value="", regex=True)

In [159]:
# Extract the day from the 'Begin First Reign', 'End First Reign', 'Begin Second Reign' and 'End Second Reign' columns.
def get_day(row_value):
    input_string = str(row_value)
    if re.search(r"^\d\s", input_string):
        day = "0" + re.findall(r"^\d\s", input_string)[0]
    elif re.search(r"^\d\d\s", input_string):
        day = re.findall(r"^\d\d\s", input_string)[0]
    else:
        day = "??"
    return day

emperors['Day Begin First Reign'] = emperors['Begin First Reign'].apply(get_day)
emperors['Day End First Reign'] = emperors['End First Reign'].apply(get_day)
emperors['Day Begin Second Reign'] = emperors['Begin Second Reign'].apply(get_day)
emperors['Day End Second Reign'] = emperors['End Second Reign'].apply(get_day)

In [160]:
# Extract the month from the 'Begin First Reign', 'End First Reign', 'Begin Second Reign' and 'End Second Reign' columns.
def get_month(row_value):
    input_string = str(row_value)
    month_dict = {'January':'01', 'February':'02', 'March':'03', 'April':'04', 'May':'05', 'June':'06',
                  'July':'07', 'August':'08', 'September':'09', 'October':'10', 'November':'11', 'December':'12'}
    month_names = "January|February|March|April|May|June|July|August|September|October|November|December"
    if re.search(month_names, input_string):
        name_of_month = re.findall(month_names, input_string)[0]
        month = month_dict[name_of_month]
    else:
        month = "??"
    return month

emperors['Month Begin First Reign'] = emperors['Begin First Reign'].apply(get_month)
emperors['Month End First Reign'] = emperors['End First Reign'].apply(get_month)
emperors['Month Begin Second Reign'] = emperors['Begin Second Reign'].apply(get_month)
emperors['Month End Second Reign'] = emperors['End Second Reign'].apply(get_month)

In [161]:
# Extract the year from the 'Begin First Reign', 'End First Reign', 'Begin Second Reign' and 'End Second Reign' columns.
def get_year(row_value):
    input_string = str(row_value)
    if re.search(r"\s\d$", input_string) or re.search(r"^\d{2}$", input_string):
        match = re.findall(r"\d$", input_string)[0]
        year = "000" + match
    elif re.search(r"\s\d{2}$", input_string) or re.search(r"^\d{2}$", input_string):
        match = re.findall(r"\d{2}$", input_string)[0]
        year = "00" + match
    elif re.search(r"\s\d{3}$", input_string) or re.search(r"^\d{3}$", input_string):
        match = re.findall(r"\d{3}$", input_string)[0]
        year = "0" + match
    elif re.search(r"\s\d{4}$", input_string) or re.search(r"^\d{4}$", input_string):
        match = re.findall(r"\d{4}$", input_string)[0]
        year = match
    else:
        year = "????"
    return year

emperors['Year Begin First Reign'] = emperors['Begin First Reign'].apply(get_year)
emperors['Year End First Reign'] = emperors['End First Reign'].apply(get_year)
emperors['Year Begin Second Reign'] = emperors['Begin Second Reign'].apply(get_year)
emperors['Year End Second Reign'] = emperors['End Second Reign'].apply(get_year)

In [162]:
# In some cases the year is missing in the 'Year Begin First Reign' and 'Year Begin Second Reign' columns
# but the year is given in the 'Year End First Reign' and 'Year End Second Reign' columns.
# Extract the missing year from the 'Year End First Reign' and 'Year End Second Reign' columns.
def missing_year(year_column_1, year_column_2):
    string_1 = str(year_column_1)
    string_2 = str(year_column_2)
    if string_1 == "????":
        year = string_2
    else:
        year = string_1
    return year

emperors['Year Begin First Reign'] = emperors.apply(
    lambda x: missing_year(x['Year Begin First Reign'], x['Year End First Reign']), axis=1)
emperors['Year Begin Second Reign'] = emperors.apply(
    lambda x: missing_year(x['Year Begin Second Reign'], x['Year End Second Reign']), axis=1)

In [163]:
# Combine the year, month, and day columns
# into new columns 'Begin First Reign', 'End First Reign', 'Begin Second Reign' and 'End Second Reign'.
emperors['Begin First Reign'] = emperors['Year Begin First Reign'].astype(str) + "-" \
        + emperors['Month Begin First Reign'].astype(str) + "-" \
        + emperors['Day Begin First Reign'].astype(str)

emperors['End First Reign'] = emperors['Year End First Reign'].astype(str) + "-" \
        + emperors['Month End First Reign'].astype(str) + "-" \
        + emperors['Day End First Reign'].astype(str)

emperors['Begin Second Reign'] = emperors['Year Begin Second Reign'].astype(str) + "-" \
        + emperors['Month Begin Second Reign'].astype(str) + "-" \
        + emperors['Day Begin Second Reign'].astype(str)

emperors['End Second Reign'] = emperors['Year End Second Reign'].astype(str) + "-" \
        + emperors['Month End Second Reign'].astype(str) + "-" \
        + emperors['Day End Second Reign'].astype(str)

In [164]:
# Mark records with missing dates for the second reign with 'No second reign'.
emperors.loc[emperors['Begin Second Reign'] == "????-??-??", 'Begin Second Reign'] = 'No second reign'
emperors.loc[emperors['End Second Reign'] == "????-??-??", 'End Second Reign'] = 'No second reign'

##### Extract level of certainty about the reign dates from the 'Reign' column 

A lot of the reign dates are uncertain. The reign columns mark the uncertainty of the dates with the label 'c.' Extract this information and store it in four new columns 'Begin First Reign', 'End First Reign', 'Begin Second Reign' and 'End Second Reign'.

In [165]:
# Extract certainty of dates.
def get_certainty(row_value):
    input_string = str(row_value)
    if re.search(r"c\.\s", input_string):
        return "approximate"
    else:
        return "certain"

emperors['Certainty Begin First Reign'] = emperors['Begin First Reign'].apply(get_certainty)
emperors['Certainty End First Reign'] = emperors['End First Reign'].apply(get_certainty)
emperors['Certainty Begin Second Reign'] = emperors['Begin Second Reign'].apply(get_certainty)
emperors['Certainty End Second Reign'] = emperors['End Second Reign'].apply(get_certainty)

In [166]:
# Remove the date label 'c.' from 'Reign' columns.
reign_columns = ['Begin First Reign', 'Begin Second Reign', 'End First Reign', 'End Second Reign']

for i in reign_columns:
    emperors[i] = emperors[i].replace(to_replace=r"^c\.\s", value="", regex=True)

##### Extract duration of first and second reign from the 'Reign' column and store it in two new columns 'Duration First Reign' and 'Duration Second Reign' 

The duration of reign is given in the years, months and days. However, the format is not uniform. First a whole string has to be extracted. Then, from that string the years, months and days have to be extracted.

In [167]:
# Extract duration of first reign.
def get_first_duration(row_value):
    input_string = str(row_value)
    if re.search(r"\(([a-zA-Z0-9\s\,\/\.\;-]*)\)", input_string):
        duration = re.findall(r"(\([a-zA-Z0-9\s\,\/\.\;-]*\))", input_string)[0]
    else:
        duration = "(30 days)"
    return duration

# Extract duration of second reign.
def get_second_duration(row_value):
    input_string = str(row_value)
    if re.search(r"\(([a-zA-Z0-9\s\,\/\.\;-]*)\)", input_string):
        duration = re.findall(r"(\([a-zA-Z0-9\s\,\/\.\;-]*\))", input_string)
        if len(duration) > 1:
            return duration[-1]
        else:
            return "No second reign"
    else:
        return "No second reign"

emperors['Duration First Reign'] = emperors['Reign'].apply(get_first_duration)
emperors['Duration Second Reign'] = emperors['Reign'].apply(get_second_duration)

Extract the years, months and the days of the duration of reign and store the information in separate columns.

In [168]:
# Extract the years of the duration of reign.
def get_duration_year(row_value):
    input_string = str(row_value)
    if re.search(r"\(\d(?=\syear)", input_string):
        year = re.findall(r"\d(?=\syear)", input_string)[0]
    elif re.search(r"\s\d(?=\syear)", input_string):
        year = re.findall(r"\d(?=\syear)", input_string)[0]
    elif re.search(r"\sa(?=\syear)", input_string):
        year = "1"
    elif re.search(r"\s\d\d(?=\syear)", input_string):
        year = re.findall(r"\d\d(?=\syear)", input_string)[0]
    elif re.search(r"\(\d\d(?=\syear)", input_string):
        year = re.findall(r"\d\d(?=\syear)", input_string)[0]
    else:
        year = "0"
    return int(year)

emperors['Years Duration First Reign'] = emperors['Duration First Reign'].apply(get_duration_year)
emperors['Years Duration Second Reign'] = emperors['Duration Second Reign'].apply(get_duration_year)

In [169]:
# Extract the months of the duration of reign.
def get_duration_month(row_value):
    input_string = str(row_value)
    if re.search(r"\(\d(?=\smonth)", input_string):
        month = re.findall(r"\d(?=\smonth)", input_string)[0]
    elif re.search(r"\s\d(?=\smonth)", input_string):
        month = re.findall(r"\d(?=\smonth)", input_string)[0]
    elif re.search(r"\sa(?=\smonth)", input_string):
        month = "1"
    elif re.search(r"\s\d\d(?=\smonth)", input_string):
        month = re.findall(r"\d\d(?=\smonth)", input_string)[0]
    elif re.search(r"\(\d\d(?=\smonth)", input_string):
        month = re.findall(r"\d\d(?=\smonth)", input_string)[0]
    else:
        month= "0"
    return int(month)

emperors['Months Duration First Reign'] = emperors['Duration First Reign'].apply(get_duration_month)
emperors['Months Duration Second Reign'] = emperors['Duration Second Reign'].apply(get_duration_month)

In [170]:
# Extract the days of the duration of reign.
def get_duration_day(row_value):
    input_string = str(row_value)
    if re.search(r"\(\d(?=\sday)", input_string):
        day = re.findall(r"\d(?=\sday)", input_string)[0]
    elif re.search(r"\s\d(?=\sday)", input_string):
        day = re.findall(r"\d(?=\sday)", input_string)[0]
    elif re.search(r"\sa(?=\sday)", input_string):
        day = "1"
    elif re.search(r"\s\d\d(?=\sday)", input_string):
        day = re.findall(r"\d\d(?=\sday)", input_string)[0]
    elif re.search(r"\(\d\d(?=\sday)", input_string):
        day = re.findall(r"\d\d(?=\sday)", input_string)[0]
    else:
        day = "0"
    return int(day)

emperors['Days Duration First Reign'] = emperors['Duration First Reign'].apply(get_duration_day)
emperors['Days Duration Second Reign'] = emperors['Duration Second Reign'].apply(get_duration_day)

The given format (years, months, days) of the duration of reign is not easy to work with. Calculate the duration in years and days.

In [171]:
# Calculate years and days of the frist reign
emperors['Days of First Reign'] = (emperors['Years Duration First Reign'] * 365)  + (emperors['Months Duration First Reign'] * 30) + emperors['Days Duration First Reign']
emperors['Years of First Reign'] = round(emperors['Days of First Reign'] / 365, 2)

# Calculate years and days of the second reign
emperors['Days of Second Reign'] = (emperors['Years Duration Second Reign'] * 365)  + (emperors['Months Duration Second Reign'] * 30) + emperors['Days Duration Second Reign']
emperors['Years of Second Reign'] = round(emperors['Days of Second Reign'] / 365, 2)

##### Extract territory information from 'Reign' column into new column 'Part of Empire'

Begining with the time of Diocletian (284-305) the Roman Empire over longer periods was split in two or more parts. Sometimes it was reunited under one emperor. Therefore emperors often controlled only parts of the Empire. The relevant information needs to be extracted from the 'Reign' column.

In [172]:
# Create a new column 'Part of Empire'.
def part_of_the_empire(reign):
    if ("Whole" in reign) or ("whole" in reign):
        return "Whole empire"
    elif "East" in reign:
        return "East"
    elif "West" in reign:
        return "West"
    elif "Italy" in reign:
        return "Italy"
    else:
        return "Whole empire"

emperors['Part of Empire'] = emperors['Reign'].apply(part_of_the_empire)

# 'Succession' column

The 'Scuccession' column contains informations about the dynastical relation of the emperors and the historical circumstances that brought the emperors to power. The several points of information are seperated by the delimiters '.' and ';'. 

##### Split the 'Succession' column 

In [173]:
# Split 'Succession' column into temporary dataframe 'sucession_split'.
succession_split = emperors['Succession'].str.split(r"(\.|;)", expand=True)

# Remove columns that only contain '.' and ';'.
succession_split = succession_split.drop(columns = succession_split.columns[(succession_split == r'.').any()])
succession_split = succession_split.drop(columns = succession_split.columns[(succession_split == r';').any()])

# Replace empty cells with 'No information'.
for i in list(succession_split.columns):
    succession_split.loc[succession_split[i].isnull(), i] = "No information"

for i in list(succession_split.columns):
    succession_split.loc[succession_split[i] == "", i] = "No information"

# Remove leading white spaces.
for i in list(succession_split.columns):
    succession_split[i] = succession_split[i].replace(to_replace=r"^\s", value="", regex=True)

# Make first letter upper case.
def first_letter_upper(row_value):
    input_string = str(row_value)
    first_letter = input_string[0].upper()
    capitalized_string = first_letter + input_string[1:]
    return capitalized_string

for i in list(succession_split.columns):
    succession_split[i] = succession_split[i].apply(first_letter_upper)

In [174]:
# Create new columns 'Succession 1', 'Succession 2', 'Succession 3' from the 'succession_split' dataframe.
index = 1
for i in list(succession_split.columns):
    emperors['Succession ' + str(index)] = succession_split[i]
    index += 1

# 'Life details' column

The 'Life details' column contains the date of birth, the date of death, the age of the emperors, as well as some biographical information, especially historical deeds of the emperors and details of the their death circumstances, given that emperors very often did not die a natural death.

All this information needs to be extracted.

##### Extract the emperors' ages

In [175]:
# Exctract age from the 'Life details' column.
def get_age(row_value):
    input_string = str(row_value)
    if re.search(r"(\d\d)(?=\))", input_string):
        age = re.findall(r"(\d\d)(?=\))", input_string)[0]
    elif re.search(r"(\s\d)(?=\))", input_string):
        age = re.findall(r"(\d)(?=\))", input_string)[0]
    else:
        age = "Unknown"
    return age

emperors['Age'] = emperors['Life details'].apply(get_age)

##### Extract date of birth and date of death

In [176]:
# Extract date of birth from the 'Life details' column.
def get_date_of_birth(row_value):
    input_string = str(row_value)
    if re.search(r"\(\?\)\s-\s", input_string):
        date = re.findall(r"(.*)(?=\(\?\)\s-\s)", input_string)[0]
    elif re.search(r"\s-\s", input_string):
        date = re.findall(r"(.*)(?=\s-\s)", input_string)[0]
    else:
        date = "Unknown"
    return date

emperors['Date of Birth'] = emperors['Life details'].apply(get_date_of_birth)


In [177]:
# Exctract date of death from the 'Life details' column.
def get_date_of_death(row_value):
    input_string = str(row_value)
    if re.search(r"\s\(a", input_string):
        date = re.findall(r"(?<=\s-\s)(.*)(?=\s\(a)", input_string)[0]
    elif re.search(r"\s-\s", input_string):
        date = re.findall(r"(?<=\s-\s)(.*)", input_string)[0]
        if re.search(r"\d\d\d[A-Z]", date):
            date = re.findall(r"(.*\d\d\d)(?=[A-Z])", date)[0]
        elif re.search(r"\(\?\)", date):
            date = re.findall(r"(.*)(?=\(\?\))", date)[0]
        elif re.search(r"\?", date):
            date = re.findall(r"(.*)(?=\?)", date)[0] 
        elif re.search(r"\s\(", date):
            date = re.findall(r"(.*)(?=\s\()", date)[0]
        else:
            date = "Unkown"
    else:
        date = "Unkown"
    return date

emperors['Date of Death'] = emperors['Life details'].apply(get_date_of_death)

##### Extract remaining information

In [178]:
# Extract the other information from the 'Life details' column into the temporary column 'Life'
def get_life_detail(row_value):
    input_string = str(row_value)
    detail = "Nothing known"
    if re.search(r".*\).*\)", input_string):
        find_string = re.findall(r"(?<=\))(.*)", input_string)[0]
        detail = re.findall(r"(?<=\))(.*)", find_string)[0]
    elif re.search(r"\(.*\)", input_string):
        detail = re.findall(r"(?<=\))(.*)", input_string)[0]
    elif re.search(r"\?[A-Za-z]", input_string): 
        detail = re.findall(r"(?<=\?)(.*)", input_string)[0]
    elif re.search(r"/\d\d\d[A-Za-z]", input_string):
        detail = re.findall(r"(?<=/\d\d\d)(.*)", input_string)[0]
    elif re.search(r"\d\d\d[A-Za-z]", input_string):
        detail = re.findall(r"(?<=\d\d\d)(.*)", input_string)[0]
    elif re.search(r"lifespan[A-Za-z]", input_string):
        detail = re.findall(r"(?<=lifespan)(.*)", input_string)[0]
    return detail

emperors['Life'] = emperors['Life details'].apply(get_life_detail)

In [179]:
# Split the 'Life' column into temporary data frame life_details_split.
life_details_split = emperors['Life'].str.split(r"(\.|;)", expand=True)

# Remove columns that only contain '.' and ';'.
life_details_split = life_details_split.drop(columns = life_details_split.columns[(life_details_split == r'.').any()])
life_details_split = life_details_split.drop(columns = life_details_split.columns[(life_details_split == r';').any()])

# Replace empty cells with 'No information'.
for i in list(life_details_split.columns):
    life_details_split.loc[life_details_split[i].isnull(), i] = "No information"

for i in list(life_details_split.columns):
    life_details_split.loc[life_details_split[i] == "", i] = "No information"

# Remove leading white spaces.
for i in list(life_details_split.columns):
    life_details_split[i] = life_details_split[i].replace(to_replace=r"^\s", value="", regex=True)

# Make first letter upper case.
def first_letter_upper(row_value):
    input_string = str(row_value)
    first_letter = input_string[0].upper()
    capitalized_string = first_letter + input_string[1:]
    return capitalized_string

for i in list(life_details_split.columns):
    life_details_split[i] = life_details_split[i].apply(first_letter_upper)


In [180]:
# Create new columns 'Life details 1', 'Life details 2', 'Life details 3', 'Life details 4' from the 'life_details_split' dataframe.
index = 1
for i in list(life_details_split.columns):
    emperors['Life details ' + str(index)] = life_details_split[i]
    index += 1

##### Extract certainty of dates of birth and death

In [181]:
# Extract certainty of dates of birth and death.
emperors['Certainty Birth'] = emperors['Date of Birth'].apply(get_certainty)
emperors['Certainty Death'] = emperors['Date of Death'].apply(get_certainty)

##### Determine the era of the date of birth

In [182]:
# Augustus was the only emperor, who was born before the year 0 (marked with 'BC').
# Determine the era of the date of birth ('BC' or 'AD') 
# and store the information in an column 'Era Birth'.
# Remove'BC' in the 'Date of Birth' column.
emperors['Era Birth'] = emperors['Date of Birth'].apply(determine_era)
emperors['Date of Birth'] = emperors['Date of Birth'].replace(to_replace=r"\sBC", value="", regex=True)

##### Create 'Date of Birth' and 'Date of Death' columns

In [183]:
# Get the year, month and day of birth and death, using the functions defined for the 'Reign' column.
emperors['Year Birth'] = emperors['Date of Birth'].apply(get_year)
emperors['Month Birth'] = emperors['Date of Birth'].apply(get_month)
emperors['Day Birth'] = emperors['Date of Birth'].apply(get_day)

emperors['Year Death'] = emperors['Date of Death'].apply(get_year)
emperors['Month Death'] = emperors['Date of Death'].apply(get_month)
emperors['Day Death'] = emperors['Date of Death'].apply(get_day)

In [184]:
# Combine the year, month, and day columns into new columns 'Date of Birth' and 'Date of Death'.
emperors['Date of Birth'] = emperors['Year Birth'].astype(str) + "-" \
        + emperors['Month Birth'].astype(str) + "-" \
        + emperors['Day Birth'].astype(str)

emperors['Date of Death'] = emperors['Year Death'].astype(str) + "-" \
        + emperors['Month Death'].astype(str) + "-" \
        + emperors['Day Death'].astype(str)

# Store end result in the emperors dataframe

In [185]:
# Keep the final columns and and drop all unnecessary columns.
emperors = emperors[[
    'Common Name', 
    'Full Name',
    'Certainty Begin First Reign',
    'Era Begin First Reign',
    'Begin First Reign',
    'Certainty End First Reign', 
    'End First Reign',
    'Days of First Reign',
    'Years of First Reign',
    'Certainty Begin Second Reign',
    'Begin Second Reign',
    'Certainty End Second Reign',
    'End Second Reign',
    'Days of Second Reign',
    'Years of Second Reign',
    'Part of Empire',
    'Period',
    'Epoch',
    'Legitimacy',
    'Status as Emperor',
    'Succession 1',
    'Succession 2',
    'Succession 3',
    'Certainty Birth',
    'Era Birth',
    'Date of Birth',
    'Certainty Death',
    'Date of Death',
    'Age',
    'Life details 1', 
    'Life details 2', 
    'Life details 3',
    'Life details 4']]

# Save the cleaned data to csv file

In [186]:
# Save cleaned data as csv-file.
emperors.to_csv(r"C:\Users\Rainer\Documents\mein github code\roman_emperors_data_cleaning\data_roman_emperors_cleaned.csv", index=False)